# SNSCrape 1

In [5]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime
import timeit
from time import sleep


maxTweets = 25
count = 0

COLUMN_NAME = ["UserID","User","Location","verified","Date_Created","Follows_Count","Friends_Count", 
        "Retweet_Count","Language","Date_Tweet","Number_of_Likes","Source_of_Tweet",
        "Tweet_Id","Tweet","Hashtags","Conversation_Id","In_reply_To","Latitude","Longitude","Place"]
df = pd.DataFrame(columns=COLUMN_NAME)

keyword = "#Pilpres2019 OR #DebatPilpres2019 OR #Pemilu2019 OR #BeraniPilih02 OR #JokowiMenangTotalDebat OR #DebatPintarJokowi OR #PrabowoMenangDebat OR #DebatKeduaPilpres2019 OR #Debat02PrabowoMenang OR #DebatCapres OR #PrabowoSandi OR #01JokowiLagi OR #2019PilihJokowi OR #2019GantiPresiden OR #01IndonesiaMaju OR #2019TetapJokowi OR #JokowiLagi OR #2019PrabowoSandi"
date = 'since:2018-09-01 until:2019-05-31 min_faves:1 exclude:retweets'

try: 
    start = timeit.default_timer()
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' ' + date).get_items()):
        if count > maxTweets:
            break
        df.loc[i] = [
                                    tweet.user.id,
                                    tweet.user.username,
                                    tweet.user.location,
                                    tweet.user.verified,
                                    tweet.user.created,
                                    tweet.user.followersCount,
                                    tweet.user.friendsCount,
                                    tweet.retweetCount,
                                    tweet.lang,
                                    tweet.date,
                                    tweet.likeCount,
                                    tweet.sourceLabel,
                                    tweet.id,
                                    tweet.content,
                                    tweet.hashtags,
                                    tweet.conversationId,
                                    tweet.inReplyToUser,
                                    tweet.coordinates.latitude if tweet.coordinates != None else None,
                                    tweet.coordinates.longitude if tweet.coordinates != None else None,
                                    tweet.Place.fullName if tweet.place != None else None]
        count += 1

    end = timeit.default_timer()
    
    print(df)
    print("\n\n\nTime elapsed: ", end - start)
except KeyboardInterrupt:
    print("KeyboardInterrupt")






C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\942034203.py:38: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\942034203.py:38: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet.content,
Skipping unrecognised entry ID: 'promoted-tweet-1658503663453716482-35946021390c9451'
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\942034203.py:38: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\942034203.py:38: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\942034203.py:38: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\

UnboundLocalError: local variable 'userId' referenced before assignment

# Location Check before mapping

In [15]:
import pandas as pd

df = pd.read_csv("Data/Pemilu_2022-11-03_08_35_19.csv")
location_count = df['Location'].value_counts()
print(location_count[1:20])


Location
DKI Jakarta, Indonesia            11801
Jakarta, Indonesia                 9792
Jakarta                            8536
Jakarta Capital Region, Indone     4282
Kota Surabaya, Jawa Timur          3556
Jakarta Capital Region             3292
Bandung, Jawa Barat                3024
Yogyakarta, Indonesia              2626
Jakarta Selatan, DKI Jakarta       2279
Jawa Barat, Indonesia              2133
Jakarta Pusat, DKI Jakarta         2109
indonesia                          2054
Aceh                               1944
Sumatera Barat, Indonesia          1798
Jawa Tengah, Indonesia             1788
Kota Medan, Sumatera Utara         1731
INDONESIA                          1652
Jakarta Timur, DKI Jakarta         1538
Denpasar Selatan, Indonesia        1382
Name: count, dtype: int64


# Mapping for Kabupaten / Kota

In [16]:
import kabupaten_kota as kk


locations = df['Location'].tolist()

def check_kabupaten_kota():
    kabupaten_kota_list = []
    for loc in locations:
        flag = False
        if loc == "":
            kabupaten_kota_list.append("Unknown")
        for prov in kk.provinsi:
            for kabkot in kk. kabupaten_kota[prov]:
                if kabkot in  str(loc).lower():
                    kabupaten_kota_list.append(kabkot)
                    flag = True
                    break
        if flag == False:
            kabupaten_kota_list.append("Unknown")
        
    return kabupaten_kota_list

kabupaten_kota_list = check_kabupaten_kota()

In [7]:
import numpy as np
kabupaten_kota_list_np = np.array(kabupaten_kota_list)
# count unique
unique, counts = np.unique(kabupaten_kota_list_np, return_counts=True)
dfcount = pd.DataFrame(counts, unique)

In [8]:
dfcount = dfcount.sort_values(by=[0], ascending=False)
dfcount.head(20)

,0
Unknown,285919
jakarta,58317
bandung,8550
bekasi,4820
surabaya,4718
bogor,4430
medan,4002
yogyakarta,3747
denpasar,2487
tangerang,2394


# Mapping for Provinsi

In [17]:
import kabupaten_kota as kk 

locations = df['Location'].tolist()

def check_provinsi():
    kabupaten_kota_list = []
    for loc in locations:
        flag = False
        if loc == "":
            kabupaten_kota_list.append("Unknown")
        for prov in kk.provinsi:
            if prov in str(loc).lower():
                kabupaten_kota_list.append(prov)
                flag = True
                break
            for kabkot in kk. kabupaten_kota[prov]:
                if kabkot in  str(loc).lower():
                    kabupaten_kota_list.append(prov)
                    flag = True
                    break
        if flag == False:
            kabupaten_kota_list.append("Unknown")


provinsi_list = check_provinsi()

In [10]:
kabupaten_kota_list_np = np.array(kabupaten_kota_list)
# count unique
unique, counts = np.unique(kabupaten_kota_list_np, return_counts=True)
dfcount = pd.DataFrame(counts, unique)
dfcount = dfcount.sort_values(by=[0], ascending=False)
dfcount.head(20)

,0
Unknown,285919
jakarta,58317
bandung,8550
bekasi,4820
surabaya,4718
bogor,4430
medan,4002
yogyakarta,3747
denpasar,2487
tangerang,2394


## Data Collection 2

In [9]:
import pandas as pd
import concurrent.futures 
import snscrape.modules.twitter as snscrape
from datetime import datetime

keyword = "#Pilpres2019 OR #DebatPilpres2019 OR #Pemilu2019 OR #BeraniPilih02 OR #JokowiMenangTotalDebat OR #DebatPintarJokowi OR #PrabowoMenangDebat OR #DebatKeduaPilpres2019 OR #Debat02PrabowoMenang OR #DebatCapres OR #PrabowoSandi OR #01JokowiLagi OR #2019PilihJokowi OR #2019GantiPresiden OR #01IndonesiaMaju OR #2019TetapJokowi OR #JokowiLagi OR #2019PrabowoSandi"
date = 'since:2018-09-01 until:2019-05-31 min_faves:1 exclude:retweets'


def get_tweets():
    results = []
    date = 'since:2018-09-01 until:2019-05-31 min_faves:1 exclude:retweets'
    for i,tweet in enumerate(snscrape.TwitterSearchScraper(keyword + " " + date).get_items()):
        results.append([tweet.user.username, tweet.user.location, tweet.id, tweet.content, tweet.renderedContent, tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.likeCount, tweet.retweetCount, tweet.quoteCount, tweet.replyCount, tweet.date])
    return results

def concurrent_get_tweets():
    count = 0
    result = []
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(get_tweets)]
        autosave = pd.DataFrame(columns=['username', 'location','tweetID', 'content', 'renderedContent', 'inReplyToTweetId', 'inReplyToUser', 'likeCount', 'retweetCount', 'quoteCount', 'replyCount', 'date'])
        for future in concurrent.futures.as_completed(futures):
            try:
                data = future.result()
                result.extend(data)
                count += 1
                if (count % 100 == 0):
                    print("Done getting ", count, " out of ", len(usernames) , " users")
                    autosave = pd.DataFrame(result, columns=['username', 'location','tweetID', 'content', 'renderedContent', 'inReplyToTweetId', 'inReplyToUser', 'likeCount', 'retweetCount', 'quoteCount', 'replyCount', 'date'])
                    autosave.to_csv('autosave_pemilu.csv', index=False)
                    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    print("Autosaved at ", now)
            except Exception as exc:
                print(exc)
                pass
    return result


res = concurrent_get_tweets()
df = pd.DataFrame(res, columns=['username', 'location','tweetID', 'content', 'renderedContent', 'inReplyToTweetId', 'inReplyToUser', 'likeCount', 'retweetCount', 'quoteCount', 'replyCount', 'date'])

C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_23448\2512939833.py:14: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  results.append([tweet.user.username, tweet.user.location, tweet.id, tweet.content, tweet.renderedContent, tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.likeCount, tweet.retweetCount, tweet.quoteCount, tweet.replyCount, tweet.date])
Skipping unrecognised entry ID: 'promoted-tweet-1658503663453716482-35a2db9e7b71f98c'
Skipping unrecognised entry ID: 'promoted-tweet-1661733639166775296-35a23cc373433793'
Skipping unrecognised entry ID: 'promoted-tweet-1658503663453716482-35a2df2f201ce636'


local variable 'userId' referenced before assignment


In [1]:
import pandas as pd
df = pd.read_csv("Data/2022-11-12_00_21_00dropped.csv")
df = df.dropna(subset=['location'])
df

,username,location,content,renderedContent,inReplyToTweetId,inReplyToUser,likeCount,retweetCount,quoteCount,replyCount,date
472,masagungt,"Yogyakarta, Indonesia",Selamat yaaa...! https://t.co/4BoSWbExkU,Selamat yaaa...! https://t.co/4BoSWbExkU,NaN,NaN,1,0,0,0,2019-04-20 03:34:48+00:00
473,masagungt,"Yogyakarta, Indonesia",@hcahyono Ternyata kitab itu masih ada 😁,@hcahyono Ternyata kitab itu masih ada 😁,1.111643e+18,https://twitter.com/hcahyono,1,0,0,0,2019-03-29 18:11:50+00:00
474,masagungt,"Yogyakarta, Indonesia",@Fahrihamzah Rajin amat merengek minta pak Jok...,@Fahrihamzah Rajin amat merengek minta pak Jok...,1.111477e+18,https://twitter.com/Fahrihamzah,5,0,0,0,2019-03-29 16:34:36+00:00
475,masagungt,"Yogyakarta, Indonesia",@Fahrihamzah @prabowo Jendral2 bintang empat e...,@Fahrihamzah @prabowo Jendral2 bintang empat e...,1.111423e+18,https://twitter.com/Fahrihamzah,1,0,0,0,2019-03-29 00:39:35+00:00
476,masagungt,"Yogyakarta, Indonesia",@Fahrihamzah Wah..akhirnya @Fahrihamzah sadar ...,@Fahrihamzah Wah..akhirnya @Fahrihamzah sadar ...,1.111425e+18,https://twitter.com/Fahrihamzah,10,0,0,1,2019-03-29 00:37:17+00:00
...,...,...,...,...,...,...,...,...,...,...,...
3956004,mentionke,"Jakarta Capital Region, Indone",@nesiayaaa @MurtadhaOne @jokowi Ability para a...,@nesiayaaa @MurtadhaOne @jokowi Ability para a...,1.048779e+18,https://twitter.com/nesiayaa,1,0,0,0,2018-10-07 03:42:10+00:00
3956005,mentionke,"Jakarta Capital Region, Indone",Jokowi juga mempelajari beberapa bahasa isyara...,Jokowi juga mempelajari beberapa bahasa isyara...,1.048778e+18,https://twitter.com/mentionke,1,0,1,1,2018-10-07 03:32:05+00:00
3956006,mentionke,"Jakarta Capital Region, Indone",Pembukaan Asian Para Games yang membuat kita t...,Pembukaan Asian Para Games yang membuat kita t...,NaN,NaN,1,0,0,0,2018-10-07 03:14:43+00:00
3956007,mentionke,"Jakarta Capital Region, Indone",Keren! Presiden @jokowi menyampaikan pidato #O...,Keren! Presiden @jokowi menyampaikan pidato #O...,NaN,NaN,2,0,0,0,2018-10-06 13:54:02+00:00


## Keywords

In [14]:
df = pd.read_csv("Data/keyword.csv", encoding='iso-8859-1', delimiter=';')
df = df.sort_values(by=['count'], ascending=False)
df.head(20)

,text,count,keyword
0,USER,372376,0
1,yg,23425,0
2,HTTPURL,21844,0
3,NaN,18236,0
4,di,16546,0
5,dan,14056,0
6,ini,11015,0
7,itu,10434,0
8,ada,8629,0
9,yang,7472,0


In [2]:
import pandas as pd
df = pd.read_csv("Data/autosave_jawa.csv")
df

,username,location,tweetID,content,renderedContent,inReplyToTweetId,inReplyToUser,likeCount,retweetCount,quoteCount,replyCount,date
0,banu_sukamto,"Jakarta, Indonesia",1134104599856418822,@berkaryainfo Pemilih #PrabowoSandi dapat prio...,@berkaryainfo Pemilih #PrabowoSandi dapat prio...,1.133703e+18,https://twitter.com/akunyangbaik,1,0,0,0,2019-05-30 14:29:41+00:00
1,banu_sukamto,"Jakarta, Indonesia",1132542330542542848,@kanseulir @DhieAjalah 🔅 Kalau media TV tidak ...,@kanseulir @DhieAjalah 🔅 Kalau media TV tidak ...,1.132295e+18,https://twitter.com/Kanseulir,23,6,0,0,2019-05-26 07:01:47+00:00
2,banu_sukamto,"Jakarta, Indonesia",1132522956574167041,@MichelAdamNew 🔅 Wujudkan #AksiKedaulatanRakya...,@MichelAdamNew 🔅 Wujudkan #AksiKedaulatanRakya...,1.132518e+18,https://twitter.com/MichelAdamNew,2,1,0,1,2019-05-26 05:44:48+00:00
3,banu_sukamto,"Jakarta, Indonesia",1130798934090104837,"@gncp_demak 🔅 Mungkin mereka lupa, sudah dilah...","@gncp_demak 🔅 Mungkin mereka lupa, sudah dilah...",NaN,NaN,2,1,1,0,2019-05-21 11:34:09+00:00
4,banu_sukamto,"Jakarta, Indonesia",1129823919702204417,@ices_tea @atoillahsmart @fadlizon @Fahrihamza...,@ices_tea @atoillahsmart @fadlizon @Fahrihamza...,NaN,NaN,1,2,0,0,2019-05-18 18:59:48+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1943844,amrullahkareem1,jakarta indonesia,1061863207181049856,@tawajkt Efendi Simbolon itu dapil Jakarta... ...,@tawajkt Efendi Simbolon itu dapil Jakarta... ...,1.061645e+18,https://twitter.com/tawajkt,7,3,0,0,2018-11-12 06:08:11+00:00
1943845,amrullahkareem1,jakarta indonesia,1056012822025392128,@ahnasaktion @teazmee Kiyai Ma'ruf udah mundur...,@ahnasaktion @teazmee Kiyai Ma'ruf udah mundur...,1.055909e+18,https://twitter.com/Teuku_Setiawan,4,0,0,1,2018-10-27 02:40:51+00:00
1943846,amrullahkareem1,jakarta indonesia,1055990899514044416,@Airin_NZ Saya sering ke Luar Batang.. Saya ta...,@Airin_NZ Saya sering ke Luar Batang.. Saya ta...,1.055733e+18,https://twitter.com/Airin_NZ,1,0,0,0,2018-10-27 01:13:44+00:00
1943847,amrullahkareem1,jakarta indonesia,1045878557728169984,"@zoelfick Cerita bu @ibutjantik ,pak Arifin sa...","@zoelfick Cerita bu @ibutjantik ,pak Arifin sa...",1.045738e+18,https://twitter.com/zoelfick,32,10,0,0,2018-09-29 03:30:54+00:00


In [11]:
jawauser = pd.read_csv("Data/jawa_user.txt", delimiter=',')
joined = df.join(jawauser.set_index('username'), on='username')
joined['provinsi'].value_counts()

provinsi
dki jakarta    1067556
jawa barat      422217
jawa timur      291467
jawa tengah     162609
Name: count, dtype: int64

In [14]:
nMax = 150000

res = joined.groupby('provinsi').apply(lambda x: x.sample(n=min(nMax, len(x))))
res['provinsi'].value_counts()

provinsi
dki jakarta    150000
jawa barat     150000
jawa tengah    150000
jawa timur     150000
Name: count, dtype: int64